In [1]:
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import random
import copy
import datetime
import time
import utm
from collections import Counter
from utm import from_latlon
# import conversion
sys.path.append('./utils')
from tqdm import tqdm
tqdm.pandas(desc="Progress")
import os
import seaborn as sns
%matplotlib inline

/Users/wuyuhan/anaconda3/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [22]:
####################################################
# poi数据处理
# 注：在高浩然学长代码上进行小改进
####################################################

In [18]:
filepath = r'../clean_data/hangzhou_pois.csv'
data = pd.read_csv(filepath,index_col=0)
data.describe()

,citycode
count,119732.000000
mean,571.027369
std,0.251401
min,571.000000
25%,571.000000
50%,571.000000
75%,571.000000
max,575.000000


In [20]:
# id去重
data.drop_duplicates(subset=['id'], keep='first', inplace=True)
# 删除多余属性，name
del data['name']
data.describe()

,citycode
count,119732.000000
mean,571.027369
std,0.251401
min,571.000000
25%,571.000000
50%,571.000000
75%,571.000000
max,575.000000


In [21]:
del data['citycode']

In [5]:
data.describe()

,id,type,location
count,119732,119732,119732
unique,119732,2310,116331
top,B023B16CAQ,商务住宅;楼宇;商务写字楼,"120.275826,30.316706"
freq,1,1105,58


In [6]:
# 把第一和第二类别抽取出来

types = data['type'].values
all_types = []
for x in types:
    k = x.split(';')
    all_types.append(k[0]+';'+k[1])
data['type'] = all_types

In [7]:
def get_types(row):
    types = row['type']
    k = types.split(';')
    return k[0], k[1]

data['type0'],data['type1'] = zip(*data.progress_apply(get_types, axis=1))
data.describe()

Progress: 100%|██████████| 119732/119732 [00:01<00:00, 106389.01it/s]


,id,type,location,type0,type1
count,119732,119732,119732,119732,119732
unique,119732,227,116331,23,227
top,B023B16CAQ,购物服务;专卖店,"120.275826,30.316706",购物服务,专卖店
freq,1,6867,58,24792,6867


In [8]:
# 发现汽车维修和汽车销售的poi需要调整

def clean_type(row):
    if (row['type0'] == '汽车维修') or (row['type0'] == '汽车销售'):
        return row['type0']
    else:
        return row['type1']
    
data['type'] = data.progress_apply(clean_type, axis=1)
data.describe()

Progress: 100%|██████████| 119732/119732 [00:01<00:00, 65809.93it/s]


,id,type,location,type0,type1
count,119732,119732,119732,119732,119732
unique,119732,158,116331,23,227
top,B023B16CAQ,专卖店,"120.275826,30.316706",购物服务,专卖店
freq,1,6867,58,24792,6867


In [9]:
# 统计一些频率过低的POI，删掉
types = data['type0'].values
c = Counter(types)
c = dict(c)
count = 0
for k, v in c.items():
    count += 1
    print(k,v)
count

汽车服务 3999
汽车销售 935
公司企业 7124
汽车维修 1104
体育休闲服务 6367
生活服务 10202
购物服务 24792
餐饮服务 12977
摩托车服务 243
住宿服务 2560
科教文化服务 6389
地名地址信息 5688
风景名胜 2065
商务住宅 4447
交通设施服务 7614
金融保险服务 3692
政府机构及社会团体 7054
医疗保健服务 6017
道路附属设施 74
公共设施 2242
事件活动 36
室内设施 900
通行设施 3211


23

In [10]:
# 统计一些频率过低的POI，删掉
types = data['type'].values
c = Counter(types)
c = dict(c)
for k, v in c.items():
    if v < 20:
        print(k,v)

加气站 15
其它能源站 5
知名企业 16
出租车 6
自来水营业厅 18
高尔夫相关 9
急救中心 18
外国机构 4
民主党派 3
交通服务相关 4
班车站 1
公司企业 14
服务区 4
道路附属设施 2
警示信息 1
市中心 1
公共设施 1
事件活动 4
特殊通道 1
虚拟门 5


In [11]:
data = data[~data.isin({'type':['加气站','其它能源站','知名企业','出租车' ,'自来水营业厅' ,'高尔夫相关' ,'急救中心' ,'外国机构' ,'民主党派' ,\
                            '交通服务相关' ,'班车站' ,'公司企业' ,'服务区' ,'道路附属设施' ,'警示信息','市中心' ,'公共设施' ,'事件活动'  ,'特殊通道','虚拟门']})]
data = data.dropna()
data.describe()

,id,type,location,type0,type1
count,119600,119600,119600,119600,119600
unique,119600,138,116201,23,207
top,B023B16CAQ,专卖店,"120.275826,30.316706",购物服务,专卖店
freq,1,6867,58,24792,6867


In [12]:
idx2type = list(set(data['type']))
type_idx = {t:i for i,t in zip(range(len(idx2type)), idx2type)}
print(idx2type)
print(type_idx)

['公检法机构', '汽车租赁', '洗浴推拿场所', '传媒机构', '冷饮店', '交通车辆管理', '商场', '科研机构', '地铁站', '政府及社会团体相关', '人才市场', '财务公司', '公共厕所', '住宿服务相关', '美术馆', '信息咨询中心', '体育休闲服务场所', '门牌信息', '汽车服务相关', '特殊买卖场所', '地名地址信息', '文化用品店', '风景名胜相关', '甜品店', '超级市场', '影剧院', '综合医院', '摩托车服务相关', '花鸟鱼虫市场', '银行相关', '紧急避难场所', '事务所', '宾馆酒店', '火车站', '洗衣店', '个人用品/化妆品店', '停车场', '外国餐厅', '摩托车销售', '便民商店/便利店', '热点地名', '自动提款机', '展览馆', '旅馆招待所', '糕饼店', '工商税务机构', '普通地名', '丧葬设施', '港口码头', '专科医院', '建筑物门', '驾校', '体育用品店', '物流速递', '自然地名', '动物医疗场所', '摄影冲印店', '商务住宅相关', '公用电话', '旅行社', '汽车配件销售', '轮渡站', '加油站', '政府机关', '邮局', '摩托车维修', '汽车救援', '报刊亭', '室内设施', '文化宫', '农林牧渔基地', '会展中心', '公交车站', '通行设施', '证券公司', '科技馆', '二手车交易', '档案馆', '餐饮相关场所', '风景名胜', '长途汽车站', '汽车维修', '工厂', '咖啡厅', '博物馆', '娱乐场所', '公园广场', '汽车俱乐部', '综合市场', '临街院门', '收费站', '茶艺馆', '学校', '楼宇', '婴儿服务场所', '医药保健销售店', '中介机构', '文艺团体', '疾病预防机构', '洗车场', '电力营业厅', '图书馆', '社会团体', '银行', '电讯营业厅', '诊所', '特色商业街', '售票处', '家居建材市场', '公司', '家电电子卖场', '休闲场所', '汽车养护/装饰', '产业园区', '机场相关', '美容美发店', '度假疗养场所', '保险公司', '专卖店', '中餐厅', 

In [13]:
# 删除没用的属性
del data['type0']
del data['type1']
del data['id']
data.describe()

,type,location
count,119600,119600
unique,138,116201
top,专卖店,"120.275826,30.316706"
freq,6867,58


In [15]:
# 把type转成idx
def type2idx(types):
    res = []
    for x in types:
        res.append(type_idx[x])
    return res
        
data['type'] = type2idx(data['type'].values)

In [16]:
location = data['location'].values
def parse_location(location):
    lons = []
    lats = []
    for x in location:
        lon, lat = x.split(',')
        lons.append(float(lon))
        lats.append(float(lat))
    return lons, lats
data['lon'], data['lat'] = parse_location(location)

data = data[(data['lon']>=120.0000) & (data['lon']<=120.4784)]
data = data[(data['lat']>=30.1334) & (data['lat']<=30.4627)]
print(data['lon'][:10])
print(data['lat'][:10])

0    120.254791
1    120.238638
2    120.207283
3    120.239136
4    120.180202
5    120.240013
6    120.224222
7    120.177645
8    120.204922
9    120.294504
Name: lon, dtype: float64
0    30.295649
1    30.298933
2    30.265033
3    30.286812
4    30.263369
5    30.292224
6    30.277566
7    30.288808
8    30.304555
9    30.299771
Name: lat, dtype: float64


In [17]:
data.describe()

,type,lon,lat
count,119394.000000,119394.000000,119394.000000
mean,79.429569,120.207289,30.284471
std,39.702077,0.081118,0.060251
min,0.000000,120.000000,30.133432
25%,43.000000,120.160445,30.251562
50%,89.000000,120.197836,30.285772
75%,115.000000,120.257107,30.318293
max,137.000000,120.478362,30.462683


In [18]:
def wgs84_2_xy(row):
    east, north, _, _ = from_latlon(row['lat'], row['lon'])#from_latlon()函数将经纬度转换成XY坐标
    return east, north

data["east"], data["north"] = zip(*data.progress_apply(wgs84_2_xy, axis=1))
data = data.drop(columns=['lon','lat']) # 删除原始的经纬度坐标
data.describe()

Progress: 100%|██████████| 119394/119394 [00:08<00:00, 13597.29it/s]


,type,east,north
count,119394.000000,119394.000000,1.193940e+05
mean,79.429569,231368.117090,3.353614e+06
std,39.702077,7815.318756,6.673807e+03
min,0.000000,211079.122959,3.336353e+06
25%,43.000000,226816.881435,3.350015e+06
50%,89.000000,230420.059748,3.353864e+06
75%,115.000000,236169.800852,3.357287e+06
max,137.000000,257610.124387,3.373519e+06


In [19]:
la = [(30.1334, 120.0000), (30.4627, 120.4784),(30.1334, 120.2784), (30.4627, 120.0000)]
for lat,lon in la:
    east, north, _, _ = from_latlon(lat, lon)
    print(east,north)

210978.74293036625 3337368.0197867323
257895.39660109906 3372760.1071537775
237810.50672122062 3336694.8923783028
211944.20813006174 3373883.5253603132


In [20]:
data['east'] = data['east'] - 210978
data['north'] = data['north'] - 3336694
data.describe()

,type,east,north
count,119394.000000,119394.000000,119394.000000
mean,79.429569,20390.117090,16919.715414
std,39.702077,7815.318756,6673.807082
min,0.000000,101.122959,-340.626291
25%,43.000000,15838.881435,13320.650297
50%,89.000000,19442.059748,17170.424172
75%,115.000000,25191.800852,20593.124471
max,137.000000,46632.124387,36824.594580


In [22]:
class Region:
    def __init__(self, east, north, e_rs, n_rs):
        self.global_width = east + 1 # 地图宽度
        self.global_height = north + 1 # 地图高度
        self.xs = e_rs # 横向区域数
        self.ys = n_rs # 纵向区域数
        self.__region_width = self.global_width / self.xs # 区域宽度        
        self.__region_height = self.global_height / self.ys # 区域高度
        self.__region_bounds = None # 每个区域的起始坐标
        self.__region_map = None # 起始坐标映射到区域
        self.__tokenization()
    def mapping(self, locations):
        """
        locations: [[e,n],...] 列表，包含东北坐标元素
        
        return: regions [region_id,...] 返回坐标所属的区域id
                xs [x,...]
                ys [y,...]
        """
        locs = np.array(locations)
        # -> n x 2
        # 向下取整
        xs=locs[:,0].reshape(-1,1) // self.__region_width
        ys=locs[:,1].reshape(-1,1) // self.__region_height
        es = xs * self.__region_width
        ns = ys * self.__region_height

        es = es.astype('int64')
        ns = ns.astype('int64')
        
        locs = np.concatenate([es,ns] ,axis = 1)
        locs = [(x,y) for x,y in locs]
        
        regions = []

        for t in locs:
            assert t[0]< self.global_width and t[1] < self.global_height, "{},{}坐标超出边界！".format(t[0],t[1])
            regions.append(self.__region_map[t])
        return regions, xs.astype('int64'), ys.astype('int64')
    
    def __tokenization(self):
        idxs = range(self.xs * self.ys)
        
        # 按照米
        x_bounds = [int(self.__region_width * i) for i in range(self.xs)]
        y_bounds = [int(self.__region_height * i) for i in range(self.ys)]
        self.__region_bounds = [(x,y) for x in x_bounds for y in y_bounds]
        self.__region_map = {bound:idx for bound,idx in zip(self.__region_bounds,idxs)}
        
        self.id2xy = [(x,y) for x in range(self.xs) for y in range(self.ys)]
        self.xy2id = {(x,y):idx for idx,(x,y) in zip(idxs,self.id2xy)}
        
    def details(self):
        print("地图宽度{},地图高度{}".format(self.global_width,self.global_height))
        print("区域宽度{}，区域高度{}".format(self.__region_width,self.__region_height))
        

In [23]:
# 计算边界，用于划分区域，正式试验数据
east_zero = 210978
north_zero = 3336694
east_max = 257895 - east_zero
north_max = 3373883 - north_zero
side_len = 32 # 一边的网格数
region = Region(east_max, north_max, side_len, side_len)
region.details()

地图宽度46918,地图高度37190
区域宽度1466.1875，区域高度1162.1875


In [24]:
data = data[(data['north'] > 0) & (data['east'] > 0)]
data

,type,location,east,north
0,62,"120.254791,30.295649",24991.320956,18044.777068
1,1,"120.238638,30.298933",23446.013025,18446.600924
2,62,"120.207283,30.265033",20336.790155,14761.662654
3,62,"120.239136,30.286812",23461.235451,17101.498625
4,62,"120.180202,30.263369",17725.730095,14641.553328
...,...,...,...,...
119722,89,"120.222641,30.217022",21684.883752,9402.115597
119723,89,"120.167382,30.271725",16514.911821,15598.764154
119724,89,"120.176562,30.273230",17402.561218,15743.636566
119725,89,"120.146966,30.287216",14593.186987,17365.605632


In [25]:
names=['id','lon','lat','state','date','time']
# dtypes = {'id' : int,'lon' : float, 'lat': float,'state' : int,'date' : int, \
#           'time' : }

In [26]:
locations = data[['east','north']].values.tolist() #locations: [[e,n],...] 列表，包含东北坐标元素
data['region'], data['x'], data['y'] = region.mapping(locations)
data = data.drop(columns=['east','north'])
data.describe()


,type,region,x,y
count,119375.000000,119375.000000,119375.000000,119375.000000
mean,79.427996,443.023765,13.405043,14.062392
std,39.703266,171.120895,5.336957,5.748466
min,0.000000,0.000000,0.000000,0.000000
25%,43.000000,335.000000,10.000000,11.000000
50%,89.000000,426.000000,13.000000,14.000000
75%,115.000000,547.000000,17.000000,17.000000
max,137.000000,1022.000000,31.000000,31.000000


In [28]:
####################################################
# 出租车数据处理
# 注：在高浩然学长代码上进行小改进
####################################################

In [192]:
# 数据装载
names=['id','lon','lat', 'state', 'date', 'time']
df = pd.read_csv('../data/taxi_raw_data/data01.txt', sep = ' ',names=names )

In [194]:
def data_cleansing(df):
    """
    Data cleansing function.
    1. Delete the invalid state.
    2. Delete the invalid time and format time 
    2. Consistent with the poi data set.
    
    Parameters:
     df - (pandas.Dataframe) Data to cleansing.
     
    Returns:
     Data (Dataframe)
    """
    df = df[df['state'].isin([1, 2])]
    df = df[df['time'].apply(lambda x : x[:x.find(':')]).astype(int) <= 24]
    df = df.dropna(axis = 0) # 删除缺失数据 
    df = df[(df['lon'] > 1.2e2) & (df['lon'] < 1.204784e2) & (df['lat'] > 3.01334e1) & (df['lat'] < 3.04627e1)]
    df['new_time'] = df['time'].apply(lambda x : ''.join(x.split(':')))
    df['date'] = df['date'].apply(lambda x : ''.join(x.split('-')))
    df['convert_time'] = df['date'] + df['new_time']
    del df['date']
    df = df[df['convert_time'].apply(lambda x : len(str(x))) <= 13]
    del df['new_time']
    return df
df = data_cleansing(df)

In [195]:
df.describe()

,lon,lat,state
count,4.624880e+06,4.624880e+06,4.624880e+06
mean,1.201689e+02,3.028237e+01,1.492861e+00
std,7.085422e-02,4.081487e-02,4.999491e-01
min,1.200000e+02,3.013342e+01,1.000000e+00
25%,1.201282e+02,3.025617e+01,1.000000e+00
50%,1.201616e+02,3.028398e+01,1.000000e+00
75%,1.201864e+02,3.031156e+01,2.000000e+00
max,1.204782e+02,3.046265e+01,2.000000e+00


In [196]:
def data_day2hours(df):
    """
    Divide the daily data to hours data.
    
    Parameters:
     df - (pandas.Dataframe) Daily data.
     
    Returns:
     Hours data (list)
 
    Raises:
     None, todo
    """
    
    hours_data = []
    # 直接截取字符串获取小时进行group
    df['hour'] = df['time'].apply(lambda x : x[ : x.find(':')]).astype(int)
    for i,g in df.groupby("hour"):  
        print(i)
        hours_data.append(g)
    return hours_data
hours_data = data_day2hours(df)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [24]:
def cal_matrix(source, region, **kwargs):
    """
    Calculate Correlation matrix(region_side_len**2, region_side_len**2)

    Parameters:
     source - (pandas.Dataframe) Hour data.
     region - (Region) Urban region object
     
    Returns:
     corr_matrix
 
    Raises:
     None, todo
    """
    side_len = region.xs
    df = source
    df = df[['id','lon','lat','state','time', 'convert_time']]
    # 墨卡托投影
    df["east"], df["north"] = zip(*df.progress_apply(wgs84_2_xy, axis=1))
    df = df.drop(columns=['lon','lat']) # 删除原始的经纬度坐标
    df['east'] = df['east'] - east_zero
    df['north'] = df['north'] - north_zero
    df=df[(df['east'] > 0) & (df['north'] > 0)]
    locations = df[['east','north']].values.tolist() #locations: [[e,n],...] 列表，包含东北坐标元素
    df['region'], df['x'], df['y'] = region.mapping(locations)
    df = df.drop(columns=['east','north'])
    # df -> (id, state, time, convert_time, x, y, region)
    print(df)
    corr_matrix = cal_corr(df, side_len)
    return corr_matrix

In [23]:
def cal_corr(source, side_len, **kwargs):
    df = copy.deepcopy(source)
    res = np.zeros((side_len**2, side_len**2))
    
    df = df.groupby(['id'])
    for id, g in df:
        data = copy.deepcopy(g)
        
        data = data.sort_values(by='convert_time',ascending=True)
        data['state_1']=data['state'].shift(1)#下移一个数
        data['change']=data['state'] - data['state_1']
        data = data[(data['change'] == 1) | (data['change'] == -1)]
        data = data.reset_index(drop=True)
        if len(data) and data.at[0,'change'] == -1: # 开头必须是上车
            data = data[1:]
                    
        if len(data) and len(data) % 2 != 0:
            data = data[0:-1]
        data = data.reset_index(drop=True)

        for i in range(0, len(data), 2):
            pick_up = data.at[i,'region']
            get_off = data.at[i+1,'region']
            res[pick_up, get_off] = res[pick_up, get_off] + 1
    return res

In [140]:
# 测试day01.txt
corr_seq = []
for i,d in enumerate(hours_data):
    print('hour',i)
    print('size', len(d))
    t0 = time.time()
    corr= cal_matrix(d, region)
    print(corr)
    corr_seq.append(corr)
#     np.save("../clean_data/" + )
    t1 = time.time()
    print('process cost', t1-t0,'s')

Progress:   1%|          | 1878/157420 [00:00<00:19, 7810.28it/s]

hour 0
size 157420
side_len 32


Progress: 100%|██████████| 157420/157420 [00:11<00:00, 13454.03it/s]


               id  state    time convert_time  region   x   y
1920380   ?AT****      1  0:0:16   2014320016     366  11  14
1920381   ?AT****      1  0:0:16   2014320016     366  11  14
1920382   ?AT****      1  0:0:16   2014320016     366  11  14
1920383   ?AT****      1  0:0:16   2014320016     366  11  14
1920384   ?AT****      1  0:0:36   2014320036     366  11  14
...           ...    ...     ...          ...     ...  ..  ..
13892933  ?JT5980      2  0:45:8   2014320458     500  15  20
13892934  ?JT5980      2  0:46:8   2014320468     500  15  20
13892936  ?JT5980      2  0:49:7   2014320497     500  15  20
13892937  ?JT5980      2  0:50:8   2014320508     500  15  20
13892938  ?JT5980      2  0:52:7   2014320527     500  15  20

[157420 rows x 7 columns]


Progress:   1%|          | 921/147964 [00:00<00:15, 9207.48it/s]

...

               id  state    time convert_time  region   x   y
1930288   ?AT0001      2  22:0:9   2014322209     550  17   6
1930289   ?AT0001      2  22:0:9   2014322209     550  17   6
1930327   ?AT0001      1  22:8:1   2014322281     516  16   4
1930328   ?AT0001      1  22:8:1   2014322281     516  16   4
1930330   ?AT0001      1  22:9:2   2014322292     516  16   4
...           ...    ...     ...          ...     ...  ..  ..
13605109  ?ATD658      1  22:9:6   2014322296     307   9  19
13605110  ?ATD658      1  22:9:6   2014322296     307   9  19
13609214  ?ATD661      1  22:0:4   2014322204     367  11  15
13609237  ?ATD661      1  22:7:0   2014322270     367  11  15
13609241  ?ATD661      2  22:8:1   2014322281     367  11  15

[15436 rows x 7 columns]


Progress:   9%|▉         | 1181/12688 [00:00<00:00, 11803.14it/s]

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
process cost 6.200116872787476 s
hour 23
size 12688
side_len 32


Progress: 100%|██████████| 12688/12688 [00:01<00:00, 12427.28it/s]


               id  state    time convert_time  region  x   y
1930518   ?AT0001      2  23:2:1   2014322321     292  9   4
1930519   ?AT0001      2  23:2:1   2014322321     292  9   4
1930521   ?AT0001      2  23:3:3   2014322333     292  9   4
1930522   ?AT0001      2  23:3:3   2014322333     292  9   4
1930524   ?AT0001      2  23:4:4   2014322344     292  9   4
...           ...    ...     ...          ...     ... ..  ..
13609435  ?ATD661      1  23:2:1   2014322321     305  9  17
13609440  ?ATD661      1  23:4:4   2014322344     305  9  17
13609445  ?ATD661      1  23:6:7   2014322367     305  9  17
13609446  ?ATD661      1  23:6:7   2014322367     305  9  17
13609447  ?ATD661      1  23:6:7   2014322367     305  9  17

[12688 rows x 7 columns]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
process cost 5.8152756690979 s


In [167]:
# 封装为函数
def process(hours_data, region):
    corr_seq = []
    for i,d in enumerate(hours_data):
        print('hour',i)
        print('size', len(d))
        t0 = time.time()
        corr= cal_matrix(d, region)
        print(corr)
        corr_seq.append(corr)
        # np.save("../clean_data/" + )
        t1 = time.time()
        print('process cost', t1-t0,'s')
    return corr_seq

In [197]:
# 测试整个文件夹
for root, dirs, files in os.walk("../data/taxi_raw_data/", topdown=False):
    del files[0]
    for name in files:
        filepath = os.path.join(root, name)
        print('##############处理'+filepath+'##################')
        print(filepath)
        # 数据装载
        names=['id','lon','lat', 'state', 'date', 'time']
        df = pd.read_csv(filepath, sep = ' ',names=names )
        # 数据清理
        df = data_cleansing(df)
        # 转换为小时数据
        hours_data = data_day2hours(df)
        corr_seq = process(hours_data, region)
        np.save('../clean_data/'+name[:name.find('.')], corr_seq)
        print('##############'+filepath+'处理文完毕##################')
        

##############处理../data/taxi_raw_data/data17.txt##################
../data/taxi_raw_data/data17.txt


Progress:   0%|          | 0/198971 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
hour 0
size 198971


Progress: 100%|██████████| 198971/198971 [00:15<00:00, 12876.39it/s]


              id  state     time  convert_time  region   x   y
2483448  ?AT****      1   0:0:12   20143170012     366  11  14
2483449  ?AT****      1   0:0:53   20143170053     366  11  14
2483450  ?AT****      1   0:1:32   20143170132     366  11  14
2483451  ?AT****      1   0:1:52   20143170152     366  11  14
2483452  ?AT****      1   0:2:13   20143170213     366  11  14
...          ...    ...      ...           ...     ...  ..  ..
7181694  ?ATD661      1  0:57:33  201431705733     305   9  17
7181695  ?ATD661      1  0:57:54  201431705754     305   9  17
7181696  ?ATD661      1  0:58:55  201431705855     305   9  17
7181697  ?ATD661      1  0:59:16  201431705916     305   9  17
7181698  ?ATD661      1  0:59:57  201431705957     305   9  17

[198971 rows x 7 columns]


...

               id  state      time   convert_time  region   x   y
5559762   ?AT****      1   22:0:57   201432622057     366  11  14
5559763   ?AT****      1   22:1:57   201432622157     366  11  14
5559764   ?AT****      1   22:2:17   201432622217     366  11  14
5559765   ?AT****      1   22:2:58   201432622258     366  11  14
5559766   ?AT****      1   22:5:17   201432622517     366  11  14
...           ...    ...       ...            ...     ...  ..  ..
11044609  ?JT5980      2  22:54:35  2014326225435     467  14  19
11044610  ?JT5980      2  22:55:35  2014326225535     467  14  19
11044611  ?JT5980      2   22:56:5   201432622565     467  14  19
11044612  ?JT5980      2  22:56:35  2014326225635     467  14  19
11044613  ?JT5980      2   22:57:5   201432622575     467  14  19

[195980 rows x 7 columns]


Progress:   1%|          | 1063/196719 [00:00<00:18, 10626.72it/s]

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
process cost 21.870265007019043 s
hour 23
size 196719


Progress: 100%|██████████| 196719/196719 [00:14<00:00, 13649.15it/s]


               id  state      time   convert_time  region   x   y
5559829   ?AT****      1   23:0:40   201432623040     366  11  14
5559830   ?AT****      1   23:1:19   201432623119     366  11  14
5559831   ?AT****      1   23:3:59   201432623359     366  11  14
5559832   ?AT****      1   23:4:39   201432623439     366  11  14
5559833   ?AT****      1   23:5:20   201432623520     366  11  14
...           ...    ...       ...            ...     ...  ..  ..
11044629  ?JT5980      2  23:11:35  2014326231135     467  14  19
11044630  ?JT5980      2   23:12:5   201432623125     467  14  19
11044631  ?JT5980      2  23:13:35  2014326231335     467  14  19
11044632  ?JT5980      2  23:16:35  2014326231635     467  14  19
11044633  ?JT5980      2   23:17:5   201432623175     467  14  19

[196719 rows x 7 columns]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
process cost 21.597257137